In [1]:
import mne
import numpy as np
import pandas as pd
import re

1: obtain epochs and evokeds representations

In [ ]:
# read info from all files.
subjList = ['A_405', 'A_408', 'B_110', 'B_309', 'B_311', 'B_316', 'C_204', 'C_429', 'E_321', 'E_415', 'E_429', 'F_027', 'F_209', 'F_210', 'G_413', 'G_428', 'H_804', 'I_719', 'I_723'] # exclude 'G_413' because it only has 3 events? and the other G only has like 30 events so like wtf :(  
conditions = ['thumb', 'index', 'middle', 'ring', 'pinkie']
fileNameList = [(subj, f'{subj}/{subj}_clean.set') for subj in subjList]
fileList = [(subj, mne.io.read_raw_eeglab(fileName, verbose=False)) for subj, fileName in fileNameList]
fileList = [(subj, file, (mne.events_from_annotations(file, verbose=False))) for subj, file in fileList]
fileList = [(subj, mne.Epochs(file, events[0], event_id=[1, 2, 3, 4, 5],tmin= 0, tmax=2, baseline=None, preload=True, verbose=False)) for subj, file, events in fileList]

fileListDict = dict(fileList)
fileList = []

for letter in ['A', 'B', 'C', 'E', 'F', 'G', 'H', 'I']: # 
    r = re.compile(f'{letter}_d*')
    subjects = list(filter(r.match, subjList))
    subjects = [fileListDict[x] for x in subjects]
    fileName = f'subj_{letter}'
    fileList.append((fileName, mne.concatenate_epochs(subjects)))

fileList

In [ ]:
subject = fileList[2]
subject[1]

In [ ]:
evokeds = {subject[1][c].average() for c in ['1', '2', '3', '4', '5']}

In [ ]:
for c in evokeds:
    c.plot_joint(title=c)

In [ ]:
file = fileList[0][1]
file.get_data(item=0).shape
file.get_data(item=0).reshape(19*401)

In [ ]:
# raw data as features/labels
all_features = {}
all_labels = {}

for subj, file in fileList:
    labels = pd.Series()
    features = pd.DataFrame(columns=[j for i in range(19) for j in range(401)])
    for trial_i in range(file.events.shape[0]):
        labels.loc[len(labels)] = file.events[trial_i][2]
        features.loc[len(features)] = file.get_data(item=trial_i).reshape(19*401) # :p i loveee hardcoding :D
    all_features[subj] = features
    all_labels[subj] = labels


for subj, file in fileList:
    all_features[subj].to_pickle(f'{subj}/data/raw_features_clean.pkl')
    all_labels[subj].to_pickle(f'{subj}/data/raw_labels_clean.pkl')

In [5]:
# load from .pkl files :D
all_features = {}
all_labels = {}
fileList = []

for letter in ['A', 'B', 'C', 'E', 'F', 'G', 'H', 'I']: 
    all_features[f'subj_{letter}'] = pd.read_pickle(f'subj_{letter}/data/raw_features_clean.pkl')
    all_labels[f'subj_{letter}'] = pd.read_pickle(f'subj_{letter}/data/raw_labels_clean.pkl')
    fileList.append((f'subj_{letter}', (all_features[f'subj_{letter}'], all_labels[f'subj_{letter}'])))


In [ ]:
print(fileList[0][1].get_data(item=trial_i)[0].T[start:end].shape)
print(fileList[0][1].get_data(item=2)[0][:][0:100].shape)

In [ ]:
# pseudosampling

window = 100 # window size
strides = [75, 50, 25, 25, 50, 75, 0] # step size (but actually it's better to make it BELL CURVE OMGGGGGGG (for overlap % maybe then...))
pseudosampled_features = {}
pseudosampled_labels = {}

for subj, file in fileList:
    labels = pd.Series()
    features = pd.DataFrame(columns=[j for i in range(19) for j in range(100)])
    for trial_i in range(file.events.shape[0]):
        start = 0
        for s in strides:
            labels.loc[len(labels)] = file.events[trial_i][2]
            end = start + window
            features.loc[len(features)] = file.get_data(item=trial_i)[0].T[start:end].reshape(19*window)
            start += s
    pseudosampled_features[subj] = features
    pseudosampled_labels[subj] = labels

            
    

In [ ]:
for subj, file in fileList:
    pseudosampled_features[subj].to_pickle(f'{subj}/data/pseudosampled_features_clean.pkl')
    pseudosampled_labels[subj].to_pickle(f'{subj}/data/pseudosampled_labels_clean.pkl')

2: FFT Time !! slay

In [ ]:
from scipy import fftpack
import matplotlib.pyplot as plt

def get_fft(file, trial, channel): # pull one piece of that data set, trial 1 of thumb data
    sample = file.get_data(item=trial, picks=channel)[0][0]
    #fig1 = plt.plot(np.linspace(0, 1.5, num=300), sample)
    # fft on that one piece...

    sig_fft = fftpack.fft(sample)
    power = np.abs(sig_fft)**2
    sample_freq = fftpack.fftfreq(sample.size)
    return power[0:int(power.size/2)]
    # fig2 = plt.plot(sample_freq[0:200], power[0:200])

In [ ]:
# do fft for everyone :)
# but acutally only try w one pereson frist ;D

sr = 200 # sample rate
t = 400 # sample size
sample_freqs = fftpack.fftfreq(t) # since all samples are the same, sample freqs are the same

all_features = {}
all_labels = {}

for subj, file in fileList:
    # file.event_id = {'thumb':1, 'index':2, 'middle':3, 'ring':4, 'pinkie':5}
    # conditions = ['thumb']
    labels = pd.Series()
    features = pd.DataFrame(columns=[j for i in range(19) for j in sample_freqs[0:int(t/2)]])
    for trial_i in range(file.events.shape[0]): #for each event
        labels.loc[len(labels)] = file.events[trial_i][2]
        features_flat = []
        for channel_i in range(19): #srry hardcoding this cuz lazy
            power = get_fft(file, trial_i, channel_i)
            features_flat.extend(power)
        features.loc[len(features)] = features_flat
    # features.to_csv(f'{subj}/fft/features.csv', index=False)
    # labels.to_csv(f'{subj}/fft/labels.csv', index=False)
    all_features[subj] = features
    all_labels[subj] = labels

#   print(f'{subj}: {file.events.shape[0]}')

STFT

In [ ]:
import scipy.signal.stft as stft

def get_stft(file, trial, channel):
    sample = file.get_data(item=trial, picks=channel)[0][0]
    f, t, Zxx = stft(sample, fs=200, window='triang', boundary= None)
    plt.pcolormesh(t, f, np.abs(Zxx), vmin=0, vmax=amp, shading='gouraud')
    plt.title('STFT Magnitude')
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.show()
    print(Zxx.shape)
    return Zxx

In [ ]:
fileList = fileList[0:1]
STFT_features = {}
STFT_labels = {}

for subj, file in fileList:
    labels = pd.Series()
    features = pd.DataFrame(columns=[j for i in range(19) for j in sample_freqs[0:int(t/2)]])
    for trial_i in range(file.events.shape[0]): #for each event
        labels.loc[len(labels)] = file.events[trial_i][2]
        features_flat = []
        for channel_i in range(19): #srry hardcoding this cuz lazy
            power = get_fft(file, trial_i, channel_i)
            features_flat.extend(power)
        features.loc[len(features)] = features_flat
    # features.to_csv(f'{subj}/fft/features.csv', index=False)
    # labels.to_csv(f'{subj}/fft/labels.csv', index=False)
    all_features[subj] = features
    all_labels[subj] = labels

1D CNN:

In [ ]:
import tensorflow as tf

class ConvNet_tf(tf.keras.Model):

  def __init__(self):
    super(ConvNet, self).__init__()
    self.conv1 = tf.keras.Sequential(
        tf.keras.layers.Conv1D(1, 32, 3, 'valid', activation='relu'),
        nn.MaxPool1D(2, 2)
    )
    
    self.conv2 = tf.keras.Sequential(
        nn.Conv1D(32, 64, 3, 'valid', activation='relu'),
        nn.MaxPool1D(2, 2)
    )

    self.hidden = tf.keras.Sequential(
        nn.Flatten(),
        nn.Dense(1600, 'relu'),
        nn.Dense(128, 'softmax')  
    )

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.conv2(x)
        x = self.hidden(x)
        return x


In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

class ConvNet_1D(nn.Module):
    def __init__(self):
        super(ConvNet_1D, self).__init__()
        ### FILL IN ### [10 POINTS]

        self.conv1 = nn.Sequential(
            nn.Conv1d(19, 32, 3),
            nn.ReLU(), 
            nn.MaxPool1d(2)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv1d(32, 64, 5),
            nn.ReLU(),
            nn.MaxPool1d(2)
        )

        self.conv3 = nn.Sequential(
            nn.Conv1d(32, 64, 7),
            nn.ReLU(),
            nn.MaxPool1d(2)
        )

        self.hidden = nn.Sequential(
            nn.Flatten(),
            # nn.Linear(121728, 256),
            nn.Linear(6208, 256),
            nn.Linear(256, 5)  
        )

    def forward(self, x):
        ### FILL IN ### [5 POINTS]
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.hidden(x)
        return x

In [34]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
# train neural net
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for subj, file in fileList[2:3]:
    features = all_features[subj]
    labels = all_labels[subj]
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size= 0.3) # 70% training 30% test
    conv_net = ConvNet_1D().to(device)
    criterion = nn.CrossEntropyLoss()
    conv_net_optimizer = torch.optim.Adam(conv_net.parameters(), lr=0.001)
    
    # Train the neural network
    for epoch in range(5):
        print("epoch = ",epoch)
        conv_net.train()
        data = X_train.to_numpy()
        targets = y_train.subtract(1, axis=0).to_numpy()
        data = torch.tensor(data.reshape(int(data.shape[0]), 1, int(data.shape[1])), dtype=torch.float32) # reshape # of trial, 1 channel, # of samples
        data = data.to(device)
        targets = torch.tensor(targets, dtype=torch.int64)
        targets = targets.to(device)

        # Forward pass
        conv_net_predictions = conv_net(data)
        conv_net_loss = criterion(conv_net_predictions, targets)

        # Backward pass
        conv_net_optimizer.zero_grad()
        conv_net_loss.backward()
        conv_net_optimizer.step()

        # Evaluate the neural network
        conv_net.eval()
        correct, total = 0, 0
        with torch.no_grad():
            data = X_train.to_numpy()
            targets = y_train.subtract(1, axis=0).to_numpy()
            data = torch.tensor(data.reshape(int(data.shape[0]), 1, int(data.shape[1])), dtype=torch.float32) # reshape # of trial, 1 channel, # of samples
            data = data.to(device)
            targets = torch.tensor(targets, dtype=torch.int64)
            targets = targets.to(device)
            conv_net_predictions = conv_net(data)
            _, predicted = torch.max(conv_net_predictions.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
        print("test accuracy = ",correct/total)

epoch =  0


RuntimeError: Given groups=1, weight of size [32, 19, 3], expected input[1314, 1, 7619] to have 19 channels, but got 1 channels instead

2d CNN

In [36]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
# train neural net
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for subj, file in fileList[3:4]:
    features = all_features[subj]
    labels = all_labels[subj]
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size= 0.3) # 70% training 30% test
    conv_net = ConvNet_1D().to(device)
    criterion = nn.CrossEntropyLoss()
    conv_net_optimizer = torch.optim.Adam(conv_net.parameters(), lr=0.001)
    
    # Train the neural network
    for epoch in range(5):
        print("epoch = ",epoch)
        conv_net.train()
        data = X_train.to_numpy()
        targets = y_train.subtract(1, axis=0).to_numpy()
        data = torch.tensor(data.reshape(int(data.shape[0]), 19, int(data.shape[1]/19)), dtype=torch.float32) # reshape # of trial, 1 channel, # of samples
        data = data.to(device)
        targets = torch.tensor(targets, dtype=torch.int64)
        targets = targets.to(device)

        # Forward pass
        conv_net_predictions = conv_net(data)
        conv_net_loss = criterion(conv_net_predictions, targets)

        # Backward pass
        conv_net_optimizer.zero_grad()
        conv_net_loss.backward()
        conv_net_optimizer.step()

        # Evaluate the neural network
        conv_net.eval()
        correct, total = 0, 0
        with torch.no_grad():
            data = X_test.to_numpy()
            targets = y_test.subtract(1, axis=0).to_numpy()
            data = torch.tensor(data.reshape(int(data.shape[0]), 19, int(data.shape[1]/19)), dtype=torch.float32) # reshape # of trial, 1 channel, # of samples
            data = data.to(device)
            targets = torch.tensor(targets, dtype=torch.int64)
            targets = targets.to(device)
            conv_net_predictions = conv_net(data)
            _, predicted = torch.max(conv_net_predictions.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
        print("test accuracy = ",correct/total)



epoch =  0
test accuracy =  0.1985611510791367
epoch =  1
test accuracy =  0.22302158273381295
epoch =  2
test accuracy =  0.23453237410071942
epoch =  3
test accuracy =  0.1553956834532374
epoch =  4
test accuracy =  0.1553956834532374


3: SVM

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics

In [ ]:
from sklearn import metrics
# SVM TIME WOOO
for subj, file in fileList:
    features = pseudosampled_features[subj]
    labels = pseudosampled_labels[subj]
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size= 0.3) # 70% training 30% test
    
    classifier = svm.SVC(kernel='rbf') # radial basis kernel; gamma should be 0.1
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)

    print(f"Accuracy {subj}:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
# SVM TIME WOOO
for subj, file in fileList:
    features = all_features[subj]
    labels = all_labels[subj]
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size= 0.3) # 70% training 30% test
    
    classifier = svm.SVC(kernel='rbf') # radial basis kernel; gamma should be 0.1
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)

    print(f"Accuracy {subj}:",metrics.accuracy_score(y_test, y_pred))

random forest ? !

In [ ]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor

for subj, file in fileList:
    features = pseudosampled_features[subj]
    labels = pseudosampled_labels[subj]
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size= 0.3) # 70% training 30% test

    rf = RandomForestRegressor(n_estimators = 1000)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    print(f"Accuracy {subj}:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
for subj, file in fileList:
    features = all_features[subj]
    labels = all_labels[subj]
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size= 0.3) # 70% training 30% test

    rf = RandomForestRegressor(n_estimators = 1000)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    
    print(f"Accuracy {subj}:",metrics.accuracy_score(y_test, y_pred))

3: try morlet??

In [ ]:
print(tfr.data.shape)
freqs.shape

In [ ]:
# Define frequencies of interest
freqs = np.logspace(*np.log10([6, 30]), num=20) 

for subj, file in fileList:
    tfr = mne.time_frequency.tfr_morlet(file, freqs=freqs, n_cycles=freqs / 2., return_itc=False)
    features = tfr.data.reshape((tfr.data.shape[0]*tfr.data.shape[1]* tfr.data.shape[2]))
    labels = [file.events[trial][2] for trial in file.events.shape[0]]  # Extract labels from the epochs
